In [70]:
import numpy as np
from scipy import signal
from pylab import *
import matplotlib.pyplot as plt
import scipy.io

In [71]:
"""
Function for initializing the graphic Equalizer

    Parameters
    ----------
    G_db : ndarray
        Command gains in dB
        
    Returns
    -------
    numsopt : ndarray
        numerator pats of the 31 filters
    densopt : ndarray
        denominator parts of the 31 filters
    fs : float
        sample frequency
    fc2 : 
        center frequencies and additional design points between them
    G_db2 : ndarray
        interpolate target gains linearly b/w command gains
    Notes
    -----
    Python reference to Liski and Välimäki 2017 DAFx-17 Paper The Quest for the Best Graphic Equalizer

"""


def initGEQ(G_db):
    fs = 44.1e3
    #fc1 = 1000 * 2 ** ( 1/3 * arange(-17,14))
    fc1 = np.array([19.69,24.80,31.25,39.37,49.61,62.50,78.75,99.21,125.0,157.5,198.4,
    250.0,315.0,396.9,500.0,630.0,793.7,1000,1260,1587,2000,2520,3175,4000,
    5040,6350,8000,10080,12700,16000,20160])
    fc2_list = [None] * (len(fc1)+len(np.sqrt(fc1[0:len(fc1)-1:1] * fc1[1::1])))
    fc2_list[::2] = fc1
    fc2_list[1::2] = np.sqrt(fc1[0:len(fc1)-1:1] * fc1[1::1])
    fc2 = np.array(fc2_list)
    wg = 2*pi*fc1/fs
    wc = 2*pi*fc2/fs
    c= 0.4
    bw = np.array((2 ** (1/3) - 2 ** (-1/3)) * wg)
    bw_adjust = 2*pi/fs
    # Bänder aus acge3.m
    #bw[16::] = [370*bw_adjust, 466.2*bw_adjust, 587.4*bw_adjust, 740.1*bw_adjust, 932.4*bw_adjust, 1175*bw_adjust, 1480*bw_adjust, 1865*bw_adjust, 2350*bw_adjust, 2846*bw_adjust, 3502*bw_adjust, 4253*bw_adjust, 5038*bw_adjust, 5689*bw_adjust, 5570*bw_adjust]
    bw[::] =[9.178*bw_adjust, 11.56*bw_adjust, 14.57*bw_adjust, 18.36*bw_adjust, 23.13*bw_adjust, 29.14*bw_adjust, 36.71*bw_adjust, 46.25*bw_adjust, 58.28*bw_adjust, 73.43*bw_adjust, 
    92.51*bw_adjust, 116.6*bw_adjust, 146.9*bw_adjust, 185.0*bw_adjust, 233.1*bw_adjust, 293.7*bw_adjust,370*bw_adjust, 466.2*bw_adjust, 587.4*bw_adjust, 740.1*bw_adjust, 932.4*bw_adjust, 1175*bw_adjust, 1480*bw_adjust, 1865*bw_adjust, 2350*bw_adjust, 2846*bw_adjust, 3502*bw_adjust, 4253*bw_adjust, 5038*bw_adjust, 5689*bw_adjust, 5570*bw_adjust]
    # Bänder aus SGE initGEQ.m 
    #bw[16::] = [369.7*bw_adjust, 465.8*bw_adjust, 586.8*bw_adjust, 739.3*bw_adjust, 930.6*bw_adjust, 1172*bw_adjust, 1476*bw_adjust, 1857*bw_adjust, 2338*bw_adjust, 2943*bw_adjust, 3704*bw_adjust, 4638*bw_adjust, 5684*bw_adjust, 6803*bw_adjust, 4117*bw_adjust]
    leak = interactionMatrix(10**(17/20)*np.ones(31),c,wg,wc,bw)

    G_db2 = np.zeros([61,1])
    G_db2[::2] = G_db
    G_db2[1::2] = (G_db[:len(G_db)-1:1]+G_db[1::1])/2
    
    Gopt_db = np.linalg.lstsq(leak.conj().T, G_db2)[0]
    Gopt = 10**(Gopt_db/20)
    
    leak2 = interactionMatrix(Gopt,c,wg,wc,bw)
    G2opt_db = np.linalg.lstsq(leak2.conj().T, G_db2)[0]
    G2opt = 10 **(G2opt_db/20)
    G2wopt_db = c * G2opt_db
    G2wopt = 10 **(G2wopt_db/20)
    
    q1 = np.zeros((1,31))    # for pareq_optimized 
    q3 = np.zeros((1,31))
    q1[0][22::] = [0.00166, 0.00295, 0.00544, 0.0105, 0.0214, 0.0456, 0.103, 0.257, 0.754]
    q3[0][22::] = [8.09e-6, 1.25e-5, 1.91e-5, 2.84e-5, 4.08e-5, 5.46e-5, 6.27e-5, 3.68e-5, -1.18e-4 ]
    
    numsopt = np.zeros((3,31))
    densopt = np.zeros((3,31))
    
    numsopt2 = np.zeros((3,31))
    densopt2 = np.zeros((3,31))
    
    for k in range(31):
        [num,den] = pareq(G2opt[k],G2wopt[k],wg[k],bw[k])
        numsopt[:,k] = num
        densopt[:,k] = den
        #gNq = q1[k]*g[k]+q3[k]*g[k]**3
        #[num2,den2] = pareq_optimized(1,10**(gNq/20),G2opt[k],G2wopt[k],wg[k],bw[k])
        #numsopt2[:,k] = num2
        #densopt2[:,k] = den2
    
    return numsopt,densopt,fs,fc2,G_db2

In [72]:
"""
Second-order parametric equalizing filter desigh with adjustable bandwidth gain

    Parameters
    ----------
    G : float64
        peak gain (linear)
    GB : float64
        bandwidth gain (linear)
    w0: float64
        center frequency (rads/sample)
    B : float64
        bandwidth (rads/sample)
    
    Returns
    -------
    num : ndarray
        numerator coefficients [b0,b1,b2]
    den : ndarray
        denominator coefficients [1,a1,a2]
        
    Notes
    -----
    Python reference to Liski and Välimäki 2017 DAFx-17 Paper The Quest for the Best Graphic Equalizer
"""
def pareq(G, GB, w0, B):
    if G == 1:
        beta = tan(B/2.)
    else: 
        beta = np.sqrt(abs(GB**2-1)/abs(G**2-GB**2))*tan(B/2)

    num = np.array([(1+G*beta), -2*cos(w0), (1-G*beta)]/(1+beta))
    den = np.array([1, -2*cos(w0)/(1+beta), (1-beta)/(1+beta)])
    
    return num, den

In [73]:
"""
Second-order parametric EQ with matching gain at Nyquist frequency

    Parameters
    ----------
    G0 : float64
        reference gain at DC (linear)
    G1 : float64
        nyquist-frequency gain
    G : float64
        peak gain (linear)
    GB : float64
        bandwidth gain (linear)
    w0: float64
        center frequency (rads/sample)
    Dw : float64
        bandwidth (rads/sample)
        
    Returns
    -------
    num : ndarray
        numerator coefficients [b0,b1,b2]
    den : ndarray
        denominator coefficients [1,a1,a2]
        
    Notes
    -----
    Python reference to Ramo, Liski and Välimäki 2020 Applied Sciences Paper Third-Octave and Bark Graphic-Equalizer Design with Symetric Band Filters
"""

def pareq_optimized(G0,G1,G,GB,w0,Dw):
    if G == 1:
        b = np.array([1,0,0])
        a = b
    
    else:
        F = np.abs(G**2 - GB**2)
        G00 = np.abs(G**2 - G0**2)
        F00 = np.abs(GB**2 - G0**2)
        
        G01 = np.abs(G**2 - G0*G1)
        G11 = np.abs(G**2 - G1**2)
        F01 = np.abs(GB**2 - G0*G1)
        F11 = np.abs(Gb**2 - G1**2)
        
        W2 = np.sqrt(G11/G00) * np.tan(w0/2)**2
        DW = (1 + np.sqrt(F00/F11) * W2) * tan(Dw/2)
        
        C = F11*DW**2-2*W2*(F01-np.sqrt(F00*F11))
        D = 2*W2*(G01-np.sqrt(G00*G11))
        
        A = np.sqrt((C*D)/F)
        B = np.sqrt((G**2*C+GB**2*D)/F)
        
        num = [(G1+G0*W2+B), -2*(G1-G0*W2), (G1-B+G0*W2)]/(1+W2+A)
        den = [1, (-2*(1-W2)), (1+W2-A)]/(1+W2+A)
        
    return num,den

In [74]:
"""
Compute the interaction matrix of a cascade GEQ

    Parameters
    ----------
    G : ndarray
        linear gain at which the leakage is determined    
    c : float
        gain factor at bandwidth (0.5 refers to db(G)/2)
    wg : ndarray
        command frequencies i.e. filter center frequencies (in rad/sample)
    wc : ndarray
        design frequencies (rad/sample) at which leakage is computed
    bw : ndarray 
        bandwidth of filters in radians
        
    Return
    ------
    leak : ndarray
        N by M matrix showing how the magnitude responses of the band filters leak to the design frequencies.
        N is determined from the length of the array wc (number of design frequencies) whereas M is 
        determined from the length of wg (number of filters)
        
    Notes
    -----
    Python reference to Liski and Välimäki 2017 DAFx-17 Paper The Quest for the Best Graphic Equalizer
"""

def interactionMatrix(G,c,wg,wc,bw):

    M = len(wg)
    N = len(wc)
    leak = np.zeros([M,N]) 
    Gdb = 20 * np.log10(G)
    Gdbw = c * Gdb
    Gw = 10 ** (Gdbw/20)
    
    for m in range(M):
        [num, den] = pareq(G[m],Gw[m],wg[m],bw[m])
        w,h = signal.freqz(num, den, wc)
        Gain = 20*np.log10(np.abs(h))/Gdb[m]
        leak[m,:] = np.abs(Gain)
    return leak

In [75]:
def plot(numsopt,densopt,fs,fc2,G_db2):
    N_freq = 2 **12
    w = np.logspace(np.log10(9),np.log10(22050), N_freq)
    H_opt = np.ones((N_freq,31), dtype=complex)
    H_opt_tot = np.ones((N_freq,1), dtype=complex)
    
    for k in range(31):
        w, h = signal.freqz(numsopt[:,k], densopt[:,k],worN=w,fs=fs)
        H_opt[:,k]= h
        H_opt_tot = H_opt[:,[k]]  * H_opt_tot
    
    
    plt.semilogx(w,20*np.log10(H_opt_tot))
    plt.plot(fc2,G_db2, "ro", markersize=4, markerfacecolor="none")
    plt.ylabel("Magnitude [dB]")
    plt.xlabel("Frequency [Hz]")
    plt.title("Frequency response")
    plt.xticks([10, 30, 100, 1000, 3000, 10000])
    plt.yticks(np.arange(-20,20,5))
    plt.grid(which="both", linestyle="--", color="grey")
    plt.show()
    
    return w,H_opt_tot,fc2,G_db2     #returns just for testing

In [76]:
def thirdOctaveGEQ():
    
    G_db = np.array([12,5,6,9,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,4,6,2])
    G_db_alternative = np.array([12,-12,-12,12,-12,-12,12,-12,-12,12,-12,-12,12,-12,-12,12,-12,-12,12,-12,-12,12,-12,-12,12,-12,-12,12,-12,-12,12])    
    
    [numsopt,densopt,fs,fc2,G_db2] = initGEQ(G_db.reshape((-1,1))) #reshape to make it 31x1 instead of 1x31
    [w,H_opt_tot,fc2,G_db2] =plot(numsopt,densopt,fs,fc2,G_db2)
    #plot(numsopt2,densopt2,fs,fc2,G_db2)
    return w,H_opt_tot,fc2,G_db2

In [77]:
[w,H_opt_tot,fc2,G_db2] = thirdOctaveGEQ()

C:/Users/Robert/AppData/Local/Temp/xpython_11540/4181854697.py:55: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  Gopt_db = np.linalg.lstsq(leak.conj().T, G_db2)[0]
C:/Users/Robert/AppData/Local/Temp/xpython_11540/52717356.py:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  num = np.array([(1+G*beta), -2*cos(w0), (1-G*beta)]/(1+beta))
C:/Users/Robert/AppData/Local/Temp/xpython_11540/52717356.py:33: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with d

In [78]:
xy_filter_mat = scipy.io.loadmat("xy_filter.mat")
xy_filter_mat.keys()
xy_filter_mat["xy_filter"]
#xy_filter_mat["xy_filter"].shape

array([[1.96900000e+01, 2.20977827e+01, 2.48000000e+01, 2.78388218e+01,
        3.12500000e+01, 3.50758108e+01, 3.93700000e+01, 4.41944080e+01,
        4.96100000e+01, 5.56832560e+01, 6.25000000e+01, 7.01560760e+01,
        7.87500000e+01, 8.83899740e+01, 9.92100000e+01, 1.11360900e+02,
        1.25000000e+02, 1.40312152e+02, 1.57500000e+02, 1.76771038e+02,
        1.98400000e+02, 2.22710575e+02, 2.50000000e+02, 2.80624304e+02,
        3.15000000e+02, 3.53586623e+02, 3.96900000e+02, 4.45477272e+02,
        5.00000000e+02, 5.61248608e+02, 6.30000000e+02, 7.07128701e+02,
        7.93700000e+02, 8.90898423e+02, 1.00000000e+03, 1.12249722e+03,
        1.26000000e+03, 1.41407921e+03, 1.58700000e+03, 1.78157234e+03,
        2.00000000e+03, 2.24499443e+03, 2.52000000e+03, 2.82860390e+03,
        3.17500000e+03, 3.56370594e+03, 4.00000000e+03, 4.48998886e+03,
        5.04000000e+03, 5.65720779e+03, 6.35000000e+03, 7.12741187e+03,
        8.00000000e+03, 8.97997773e+03, 1.00800000e+04, 1.131441

In [79]:
xy_semilog_mat = scipy.io.loadmat("xy_semilog.mat")
xy_semilog_mat.keys()
xy_semilog_mat["xy_semilog"]

array([[9.00000000e+00, 9.01716766e+00, 9.03436806e+00, ...,
        2.19661186e+04, 2.20080193e+04, 2.20500000e+04],
       [7.51864219e-01, 7.55742533e-01, 7.59645292e-01, ...,
        2.08174505e-02, 5.24849351e-03, 0.00000000e+00]])

In [80]:
xy_semilog_mat["xy_semilog"][0] - w

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
       4.72937245e-11, 4.36557457e-11, 4.36557457e-11])

In [99]:
xy_semilog_mat["xy_semilog"][1]

array([0.75186422, 0.75574253, 0.75964529, ..., 0.02081745, 0.00524849,
       0.        ])

In [105]:
20*np.log10(np.abs(H_opt_tot)).reshape((1,-1))

array([[ 7.51864220e-01,  7.55742533e-01,  7.59645292e-01, ...,
         2.08174505e-02,  5.24849351e-03, -1.06076021e-14]])

In [106]:
xy_semilog_mat["xy_semilog"][1]- 20*np.log10(np.abs(H_opt_tot)).reshape((1,-1))

array([[-5.01969688e-10,  1.81725079e-10, -2.60657051e-11, ...,
        -1.18623861e-13, -2.81077245e-14,  1.06076021e-14]])

In [82]:
xy_filter_mat["xy_filter"][0] - fc2

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [98]:
xy_filter_mat["xy_filter"][1] - G_db2.reshape((1,-1))

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])